In [1]:
import pandas as pd

#### Q1. Downloading the data
We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2022.

Read the data for January. How many columns are there?

Ans: 19

In [2]:
jan_2022_df = pd.read_parquet('./data/yellow_tripdata_2022-01.parquet')
feb_2022_df = pd.read_parquet('./data/yellow_tripdata_2022-02.parquet')

In [3]:
print(f"Jan 2022 dataset's columns count is {len(jan_2022_df.columns)}")
print(f"Feb 2022 dataset's columns count is {len(feb_2022_df.columns)}")

Jan 2022 dataset's columns count is 19
Feb 2022 dataset's columns count is 19


#### Q2. Computing duration
Now let's compute the `duration` variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

Ans: 46.45

In [4]:
jan_2022_df['duration'] = (jan_2022_df['tpep_dropoff_datetime'] - jan_2022_df['tpep_pickup_datetime']) / pd.Timedelta(minutes=1)
feb_2022_df['duration'] = (feb_2022_df['tpep_dropoff_datetime'] - feb_2022_df['tpep_pickup_datetime']) / pd.Timedelta(minutes=1)

In [5]:
jan_2022_df['duration'].std()

46.44530513776802

In [6]:
feb_2022_df['duration'].std()

47.26394076778834

#### Q3. Dropping outliers
Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

Ans: 98%

In [7]:
def trip_duration_between_1_to_60_mins(df):
    return df['duration'].between(1, 60, inclusive='both')

jan_2022_cleaned_df = jan_2022_df[trip_duration_between_1_to_60_mins(jan_2022_df)]
feb_2022_cleaned_df = feb_2022_df[trip_duration_between_1_to_60_mins(feb_2022_df)]

In [8]:
len(jan_2022_cleaned_df) / len(jan_2022_df)

0.9827547930522406

In [9]:
len(feb_2022_cleaned_df) / len(feb_2022_df)

0.979444397269143

#### Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

* Turn the dataframe into a list of dictionaries
* Fit a dictionary vectorizer
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

Ans: 515

In [10]:
from sklearn.feature_extraction import DictVectorizer

feature_columns = ['PULocationID', 'DOLocationID']
jan_2022_feature_dict =  jan_2022_cleaned_df[feature_columns].astype(str).to_dict(orient='records')
feb_2022_feature_dict =  feb_2022_cleaned_df[feature_columns].astype(str).to_dict(orient='records')

dv = DictVectorizer()

jan_2022_feature_vector = dv.fit_transform(jan_2022_feature_dict)
feb_2022_feature_vector = dv.transform(feb_2022_feature_dict)

In [11]:
jan_2022_feature_vector.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [12]:
feb_2022_feature_vector.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

#### Q5. Training a model

Now let's use the feature matrix from the previous step to train a model.

- Train a plain linear regression model with default parameters
- Calculate the RMSE of the model on the training data

What's the RMSE on train?

Ans: 6.99

In [13]:
label_column = 'duration'
jan_2022_label = jan_2022_cleaned_df[label_column].values
feb_2022_label = feb_2022_cleaned_df[label_column].values

In [14]:
from sklearn.metrics import root_mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso, Ridge

lr = LinearRegression()
lr.fit(jan_2022_feature_vector, jan_2022_label)

prediction = lr.predict(jan_2022_feature_vector)

rmse = root_mean_squared_error(jan_2022_label, prediction)

In [15]:
rmse

6.986191236222312

#### Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2022).

What's the RMSE on validation?

Ans: 7.79

In [16]:
feb_prediction = lr.predict(feb_2022_feature_vector)

feb_rmse = root_mean_squared_error(feb_2022_label, feb_prediction)

In [17]:
feb_rmse

7.78641213132457

In [18]:
import pickle

with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [19]:
# Save in the format needed by homework #4
with open('models/model.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)